# Generate Dataset

The dataset generation phase was designed to provide a diverse and representative set of power flow scenarios that capture both nominal operating conditions and variations arising from topological changes. To this end, we employed the IEEE test network as the baseline model and systematically simulated a wide range of load and generation profiles. Each observation in the dataset consists of nodal features, such as active power 
(
𝑃
)
(P), reactive power 
(
𝑄
)
(Q), voltage magnitude 
(
𝑉
)
(V), and voltage angle 
(
𝜃
)
(θ), along with node type information (Slack, PV, PQ). Additionally, branch-related parameters, including line conductance 
(
𝐺
)
(G), susceptance 
(
𝐵
)
(B), and line type, were incorporated to preserve the electrical properties of the network.

To enhance the generalization capability of the proposed models, topological perturbations were introduced through controlled modifications in the adjacency matrix. These perturbations emulate realistic contingencies such as line disconnections and variations in connectivity, thereby creating multiple graph representations for the same physical system. For each perturbed state, we ensured that the resulting dataset retained electrical feasibility by running load flow calculations and validating that operating constraints were respected.

The final dataset comprises both unperturbed and perturbed network configurations, enabling the learning process to account for a wide spectrum of operating conditions. This approach not only improves robustness against unseen topological variations but also better reflects the stochastic and dynamic nature of modern power distribution systems.

### Librerias

In [1]:
import pandapower as pp
import pandapower.networks as nw
import pandapower.topology as top
import networkx as nx
import numpy as np
import pandas as pd
import copy, random
import numba
print(numba.__version__)


0.61.2


## Test Case 14 Buses - Panda Power

In [1]:
import pandapower as pp
import pandapower.networks as nw
import pandapower.topology as top
import networkx as nx
import numpy as np
import pandas as pd
import copy, random

# Configuración inicial
net_base = nw.case14()
orig_p, orig_q = net_base.load.p_mw.copy(), net_base.load.q_mvar.copy()
VAR_RANGE = 0.4

# Funciones necesarias
def is_connected(net):
    G = top.create_nxgraph(net)
    return nx.is_connected(G)

def eligible_nodes(net, min_degree):
    G = top.create_nxgraph(net)
    return [node for node, deg in G.degree if deg > min_degree]

def lines_connected_to_node(net, node):
    return net.line.index[(net.line.from_bus == node) | (net.line.to_bus == node)].tolist()

def generate_sample(net, augment=True, max_attempts=20):
    attempts = 0
    while attempts < max_attempts and augment:
        net_trial = copy.deepcopy(net)
        k = random.choice([1, 2, 3])
        eligible = eligible_nodes(net_trial, k)

        if not eligible:
            attempts += 1
            continue

        node_selected = random.choice(eligible)
        candidate_lines = lines_connected_to_node(net_trial, node_selected)

        if len(candidate_lines) < k:
            attempts += 1
            continue

        lines_off = random.sample(candidate_lines, k)
        net_trial.line.loc[lines_off, 'in_service'] = False

        if is_connected(net_trial):
            try:
                pp.runpp(net_trial)
                return net_trial, lines_off
            except pp.LoadflowNotConverged:
                pass
        attempts += 1

    if not augment:
        net_trial = copy.deepcopy(net)
        pp.runpp(net_trial)
        return net_trial, []

    return None, []

# Generador de registros
def build_records(net, n_samples, augment):
    records = []
    sample_count = 0
    while sample_count < n_samples:
        net_copy = copy.deepcopy(net)
        net_copy.load['p_mw'] = orig_p * (1 + np.random.uniform(-VAR_RANGE, VAR_RANGE, len(orig_p)))
        net_copy.load['q_mvar'] = orig_q * (1 + np.random.uniform(-VAR_RANGE, VAR_RANGE, len(orig_q)))

        net_var, lines_off = generate_sample(net_copy, augment)

        if net_var is None:
            continue

        Z_vector = np.ones(net.bus.shape[0], dtype=int)
        for idx in lines_off:
            Z_vector[net_var.line.at[idx, 'from_bus']] = 0
            Z_vector[net_var.line.at[idx, 'to_bus']] = 0

        rec = {
            "Sample": f"Data {sample_count+1}",
            "Label": "FAIL" if lines_off else "OK",
            "Edges_Out_List": [f"{net.line.at[idx, 'from_bus']}-{net.line.at[idx, 'to_bus']}" for idx in lines_off],
            "Branch_Edge_Index": [[], []],
            "Branch_G": [], "Branch_B": [], "Branch_Type": [],
            "Z_vector": Z_vector.tolist()
        }

        for _, ln in net_var.line[net_var.line.in_service].iterrows():
            u, v = int(ln.from_bus), int(ln.to_bus)
            z = complex(ln.r_ohm_per_km * ln.length_km, ln.x_ohm_per_km * ln.length_km)
            y = 1/z
            rec["Branch_Edge_Index"][0].append(u)
            rec["Branch_Edge_Index"][1].append(v)
            rec["Branch_G"].append(y.real)
            rec["Branch_B"].append(y.imag)
            rec["Branch_Type"].append(0)

        for _, tr in net_var.trafo[net_var.trafo.in_service].iterrows():
            u, v = int(tr.hv_bus), int(tr.lv_bus)
            r_pu = tr.vkr_percent / 100.0
            vk = tr.vk_percent / 100.0
            x_pu = np.sqrt(max(vk*vk - r_pu*r_pu, 0.0))
            y = 1 / complex(r_pu, x_pu)
            rec["Branch_Edge_Index"][0].append(u)
            rec["Branch_Edge_Index"][1].append(v)
            rec["Branch_G"].append(y.real)
            rec["Branch_B"].append(y.imag)
            rec["Branch_Type"].append(1)

        for bus in net.bus.index:
            p_load = net_var.res_load.p_mw[net_var.load.bus == bus].sum() if bus in net_var.load.bus.values else 0
            q_load = net_var.res_load.q_mvar[net_var.load.bus == bus].sum() if bus in net_var.load.bus.values else 0
            ntype = "PQ" if bus in net_var.load.bus.values else ("Slack" if net_var.bus.at[bus, 'type']==3 else "PV")
            rec[f"P_{bus} ({ntype})"], rec[f"Q_{bus} ({ntype})"] = p_load, q_load
            rec[f"V_{bus}"] = net_var.res_bus.vm_pu.at[bus]
            rec[f"d_{bus}"] = net_var.res_bus.va_degree.at[bus]
            rec[f"Z_{bus}"] = int(Z_vector[bus])

        records.append(rec)
        sample_count += 1
        print(f"Generadas: {sample_count}/{n_samples}", end='\r')

    return records

# ─── Generación y mezcla total ───────────────────────────────────────────────
all_records = (
    build_records(net_base, 10000, augment=False) +
    build_records(net_base, 5000, augment=True) +
    build_records(net_base, 10000, augment=True) +
    build_records(net_base, 5000, augment=False)
)

df_all = pd.DataFrame(all_records).sample(frac=1, random_state=42).reset_index(drop=True)

# ─── Split 70% train / 15% val / 15% test ─────────────────────────────────────
n_total = len(df_all)
n_train = int(0.70 * n_total)
n_val   = int(0.15 * n_total)
n_test  = n_total - n_train - n_val

train_df = df_all.iloc[:n_train]
val_df   = df_all.iloc[n_train:n_train + n_val]
test_df  = df_all.iloc[n_train + n_val:]

# ─── Guardar resultados ──────────────────────────────────────────────────────
train_df.to_excel("train_data_final_14_buses.xlsx", index=False)
val_df.to_excel("val_data_final_14_buses.xlsx", index=False)
test_df.to_excel("test_data_final_14_buses.xlsx", index=False)

print("\n✅ Datasets generados y divididos correctamente:")
print(f"  Train → {len(train_df)} muestras")
print(f"  Val   → {len(val_df)} muestras")
print(f"  Test  → {len(test_df)} muestras")

Generadas: 5000/500000
✅ Datasets generados y divididos correctamente:
  Train → 21000 muestras
  Val   → 4500 muestras
  Test  → 4500 muestras


## Test Case 30 Buses - Panda Power

In [1]:
import pandapower as pp
import pandapower.networks as nw
import pandapower.topology as top
import networkx as nx
import numpy as np
import pandas as pd
import copy, random

# Configuración inicial
net_base = nw.case30()
orig_p, orig_q = net_base.load.p_mw.copy(), net_base.load.q_mvar.copy()
VAR_RANGE = 0.4

# Funciones necesarias
def is_connected(net):
    G = top.create_nxgraph(net)
    return nx.is_connected(G)

def eligible_nodes(net, min_degree):
    G = top.create_nxgraph(net)
    return [node for node, deg in G.degree if deg > min_degree]

def lines_connected_to_node(net, node):
    return net.line.index[(net.line.from_bus == node) | (net.line.to_bus == node)].tolist()

def generate_sample(net, augment=True, max_attempts=20):
    attempts = 0
    while attempts < max_attempts and augment:
        net_trial = copy.deepcopy(net)
        k = random.choice([1, 2, 3])
        eligible = eligible_nodes(net_trial, k)

        if not eligible:
            attempts += 1
            continue

        node_selected = random.choice(eligible)
        candidate_lines = lines_connected_to_node(net_trial, node_selected)

        if len(candidate_lines) < k:
            attempts += 1
            continue

        lines_off = random.sample(candidate_lines, k)
        net_trial.line.loc[lines_off, 'in_service'] = False

        if is_connected(net_trial):
            try:
                pp.runpp(net_trial)
                return net_trial, lines_off
            except pp.LoadflowNotConverged:
                pass
        attempts += 1

    if not augment:
        net_trial = copy.deepcopy(net)
        pp.runpp(net_trial)
        return net_trial, []

    return None, []

# Generador de registros
def build_records(net, n_samples, augment):
    records = []
    sample_count = 0
    while sample_count < n_samples:
        net_copy = copy.deepcopy(net)
        net_copy.load['p_mw'] = orig_p * (1 + np.random.uniform(-VAR_RANGE, VAR_RANGE, len(orig_p)))
        net_copy.load['q_mvar'] = orig_q * (1 + np.random.uniform(-VAR_RANGE, VAR_RANGE, len(orig_q)))

        net_var, lines_off = generate_sample(net_copy, augment)

        if net_var is None:
            continue

        Z_vector = np.ones(net.bus.shape[0], dtype=int)
        for idx in lines_off:
            Z_vector[net_var.line.at[idx, 'from_bus']] = 0
            Z_vector[net_var.line.at[idx, 'to_bus']] = 0

        rec = {
            "Sample": f"Data {sample_count+1}",
            "Label": "FAIL" if lines_off else "OK",
            "Edges_Out_List": [f"{net.line.at[idx, 'from_bus']}-{net.line.at[idx, 'to_bus']}" for idx in lines_off],
            "Branch_Edge_Index": [[], []],
            "Branch_G": [], "Branch_B": [], "Branch_Type": [],
            "Z_vector": Z_vector.tolist()
        }

        for _, ln in net_var.line[net_var.line.in_service].iterrows():
            u, v = int(ln.from_bus), int(ln.to_bus)
            z = complex(ln.r_ohm_per_km * ln.length_km, ln.x_ohm_per_km * ln.length_km)
            y = 1/z
            rec["Branch_Edge_Index"][0].append(u)
            rec["Branch_Edge_Index"][1].append(v)
            rec["Branch_G"].append(y.real)
            rec["Branch_B"].append(y.imag)
            rec["Branch_Type"].append(0)

        for _, tr in net_var.trafo[net_var.trafo.in_service].iterrows():
            u, v = int(tr.hv_bus), int(tr.lv_bus)
            r_pu = tr.vkr_percent / 100.0
            vk = tr.vk_percent / 100.0
            x_pu = np.sqrt(max(vk*vk - r_pu*r_pu, 0.0))
            y = 1 / complex(r_pu, x_pu)
            rec["Branch_Edge_Index"][0].append(u)
            rec["Branch_Edge_Index"][1].append(v)
            rec["Branch_G"].append(y.real)
            rec["Branch_B"].append(y.imag)
            rec["Branch_Type"].append(1)

        for bus in net.bus.index:
            p_load = net_var.res_load.p_mw[net_var.load.bus == bus].sum() if bus in net_var.load.bus.values else 0
            q_load = net_var.res_load.q_mvar[net_var.load.bus == bus].sum() if bus in net_var.load.bus.values else 0
            ntype = "PQ" if bus in net_var.load.bus.values else ("Slack" if net_var.bus.at[bus, 'type']==3 else "PV")
            rec[f"P_{bus} ({ntype})"], rec[f"Q_{bus} ({ntype})"] = p_load, q_load
            rec[f"V_{bus}"] = net_var.res_bus.vm_pu.at[bus]
            rec[f"d_{bus}"] = net_var.res_bus.va_degree.at[bus]
            rec[f"Z_{bus}"] = int(Z_vector[bus])

        records.append(rec)
        sample_count += 1
        print(f"Generadas: {sample_count}/{n_samples}", end='\r')

    return records

# ─── Generación y mezcla total ───────────────────────────────────────────────
all_records = (
    build_records(net_base, 10000, augment=False) +
    build_records(net_base, 5000, augment=True) +
    build_records(net_base, 10000, augment=True) +
    build_records(net_base, 5000, augment=False)
)

df_all = pd.DataFrame(all_records).sample(frac=1, random_state=42).reset_index(drop=True)

# ─── Split 70% train / 15% val / 15% test ─────────────────────────────────────
n_total = len(df_all)
n_train = int(0.70 * n_total)
n_val   = int(0.15 * n_total)
n_test  = n_total - n_train - n_val

train_df = df_all.iloc[:n_train]
val_df   = df_all.iloc[n_train:n_train + n_val]
test_df  = df_all.iloc[n_train + n_val:]

# ─── Guardar resultados ──────────────────────────────────────────────────────
train_df.to_excel("train_data_final_30_buses.xlsx", index=False)
val_df.to_excel("val_data_final_30_buses.xlsx", index=False)
test_df.to_excel("test_data_final_30_buses.xlsx", index=False)

print("\n✅ Datasets generados y divididos correctamente:")
print(f"  Train → {len(train_df)} muestras")
print(f"  Val   → {len(val_df)} muestras")
print(f"  Test  → {len(test_df)} muestras")

Generadas: 5000/500000
✅ Datasets generados y divididos correctamente:
  Train → 21000 muestras
  Val   → 4500 muestras
  Test  → 4500 muestras


## Test Case 118 Buses - Panda Power

In [4]:
import pandapower as pp
import pandapower.networks as nw
import pandapower.topology as top
import networkx as nx
import numpy as np
import pandas as pd
import copy, random

# Configuración inicial
net_base = nw.case118()
orig_p, orig_q = net_base.load.p_mw.copy(), net_base.load.q_mvar.copy()
VAR_RANGE = 0.4

# Funciones necesarias
def is_connected(net):
    G = top.create_nxgraph(net)
    return nx.is_connected(G)

def eligible_nodes(net, min_degree):
    G = top.create_nxgraph(net)
    return [node for node, deg in G.degree if deg > min_degree]

def lines_connected_to_node(net, node):
    return net.line.index[(net.line.from_bus == node) | (net.line.to_bus == node)].tolist()

def generate_sample(net, augment=True, max_attempts=20):
    attempts = 0
    while attempts < max_attempts and augment:
        net_trial = copy.deepcopy(net)
        k = random.choice([1, 2, 3])
        eligible = eligible_nodes(net_trial, k)

        if not eligible:
            attempts += 1
            continue

        node_selected = random.choice(eligible)
        candidate_lines = lines_connected_to_node(net_trial, node_selected)

        if len(candidate_lines) < k:
            attempts += 1
            continue

        lines_off = random.sample(candidate_lines, k)
        net_trial.line.loc[lines_off, 'in_service'] = False

        if is_connected(net_trial):
            try:
                pp.runpp(net_trial)
                return net_trial, lines_off
            except pp.LoadflowNotConverged:
                pass
        attempts += 1

    if not augment:
        net_trial = copy.deepcopy(net)
        pp.runpp(net_trial)
        return net_trial, []

    return None, []

# Generador de registros
def build_records(net, n_samples, augment):
    records = []
    sample_count = 0
    while sample_count < n_samples:
        net_copy = copy.deepcopy(net)
        net_copy.load['p_mw'] = orig_p * (1 + np.random.uniform(-VAR_RANGE, VAR_RANGE, len(orig_p)))
        net_copy.load['q_mvar'] = orig_q * (1 + np.random.uniform(-VAR_RANGE, VAR_RANGE, len(orig_q)))

        net_var, lines_off = generate_sample(net_copy, augment)

        if net_var is None:
            continue

        Z_vector = np.ones(net.bus.shape[0], dtype=int)
        for idx in lines_off:
            Z_vector[net_var.line.at[idx, 'from_bus']] = 0
            Z_vector[net_var.line.at[idx, 'to_bus']] = 0

        rec = {
            "Sample": f"Data {sample_count+1}",
            "Label": "FAIL" if lines_off else "OK",
            "Edges_Out_List": [f"{net.line.at[idx, 'from_bus']}-{net.line.at[idx, 'to_bus']}" for idx in lines_off],
            "Branch_Edge_Index": [[], []],
            "Branch_G": [], "Branch_B": [], "Branch_Type": [],
            "Z_vector": Z_vector.tolist()
        }

        for _, ln in net_var.line[net_var.line.in_service].iterrows():
            u, v = int(ln.from_bus), int(ln.to_bus)
            z = complex(ln.r_ohm_per_km * ln.length_km, ln.x_ohm_per_km * ln.length_km)
            y = 1/z
            rec["Branch_Edge_Index"][0].append(u)
            rec["Branch_Edge_Index"][1].append(v)
            rec["Branch_G"].append(y.real)
            rec["Branch_B"].append(y.imag)
            rec["Branch_Type"].append(0)

        for _, tr in net_var.trafo[net_var.trafo.in_service].iterrows():
            u, v = int(tr.hv_bus), int(tr.lv_bus)
            r_pu = tr.vkr_percent / 100.0
            vk = tr.vk_percent / 100.0
            x_pu = np.sqrt(max(vk*vk - r_pu*r_pu, 0.0))
            y = 1 / complex(r_pu, x_pu)
            rec["Branch_Edge_Index"][0].append(u)
            rec["Branch_Edge_Index"][1].append(v)
            rec["Branch_G"].append(y.real)
            rec["Branch_B"].append(y.imag)
            rec["Branch_Type"].append(1)

        for bus in net.bus.index:
            p_load = net_var.res_load.p_mw[net_var.load.bus == bus].sum() if bus in net_var.load.bus.values else 0
            q_load = net_var.res_load.q_mvar[net_var.load.bus == bus].sum() if bus in net_var.load.bus.values else 0
            ntype = "PQ" if bus in net_var.load.bus.values else ("Slack" if net_var.bus.at[bus, 'type']==3 else "PV")
            rec[f"P_{bus} ({ntype})"], rec[f"Q_{bus} ({ntype})"] = p_load, q_load
            rec[f"V_{bus}"] = net_var.res_bus.vm_pu.at[bus]
            rec[f"d_{bus}"] = net_var.res_bus.va_degree.at[bus]
            rec[f"Z_{bus}"] = int(Z_vector[bus])

        records.append(rec)
        sample_count += 1
        print(f"Generadas: {sample_count}/{n_samples}", end='\r')

    return records

# ─── Generación y mezcla total ───────────────────────────────────────────────
all_records = (
    build_records(net_base, 10000, augment=False) +
    build_records(net_base, 5000, augment=True) +
    build_records(net_base, 10000, augment=True) +
    build_records(net_base, 5000, augment=False)
)

df_all = pd.DataFrame(all_records).sample(frac=1, random_state=42).reset_index(drop=True)

# ─── Split 70% train / 15% val / 15% test ─────────────────────────────────────
n_total = len(df_all)
n_train = int(0.70 * n_total)
n_val   = int(0.15 * n_total)
n_test  = n_total - n_train - n_val

train_df = df_all.iloc[:n_train]
val_df   = df_all.iloc[n_train:n_train + n_val]
test_df  = df_all.iloc[n_train + n_val:]

# ─── Guardar resultados ──────────────────────────────────────────────────────
train_df.to_excel("train_data_final_118_buses.xlsx", index=False)
val_df.to_excel("val_data_final_118_buses.xlsx", index=False)
test_df.to_excel("test_data_final_118_buses.xlsx", index=False)

print("\n✅ Datasets generados y divididos correctamente:")
print(f"  Train → {len(train_df)} muestras")
print(f"  Val   → {len(val_df)} muestras")
print(f"  Test  → {len(test_df)} muestras")

Generadas: 5000/500000
✅ Datasets generados y divididos correctamente:
  Train → 21000 muestras
  Val   → 4500 muestras
  Test  → 4500 muestras
